In [39]:
#Import Dependencies

from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np

from time import sleep

In [27]:
# Sets a path to Google Chrome
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [/Users/joescuteri/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver] found in cache


In [40]:
URL = ['https://twitchtracker.com/games', 'https://twitchtracker.com/games?page=2', 'https://twitchtracker.com/games?page=3', 'https://twitchtracker.com/games?page=4', 'https://twitchtracker.com/games?page=5']
tags = []
games = []
cleanedviews = []
views = []

for url in range(0,5):      
    browser.visit(URL[url])
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    href_tags = soup.find_all(href=True)
    raw_games = href_tags[40:60]
    
    for x in raw_games:
        tags.append(str(x))
    
    for element in tags:
        if element not in games:
            front = element.split(">")[1]
            back = front.split("<")[0]
            games.append(back)
        
    viewhtml = soup.find_all('div', class_="ri-value")
    
    for x in viewhtml:
        cleanedviews.append(str(x))
    

    for element in cleanedviews:
        if element not in views:
            front = element.split(">")[2]
            back = front.split("<")[0]
            views.append(back)
            
    print(f'Page Done!')
    
    sleep(5)

Page Done!
Page Done!
Page Done!
Page Done!
Page Done!


In [41]:
# Upload to DataFrame 
df = pd.DataFrame(games, columns = ['GAME'])
df['VIEWS'] = views
df

,GAME,VIEWS
0,League of Legends,376K
1,Grand Theft Auto V,285K
2,VALORANT,211K
3,Rust,128K
4,Fortnite,122K
...,...,...
295,"Animals, Aquariums, and Zoos",3.0K
296,TrackMania,3.0K
297,Tibia,2.9K
298,Raft,2.9K


In [42]:
# Change VIEWS column to integers

df.VIEWS = (df.VIEWS.replace(r'[KM]+$', '', regex=True).astype(float) * df.VIEWS.str.extract(r'[\d\.]+([KM]+)', expand=False)
             .fillna(1)
             .replace(['K','M'], [10**3, 10**6]).astype(int))

In [43]:
avg_views = df['VIEWS'].mean()
median_views = df['VIEWS'].median()

In [44]:
df

,GAME,VIEWS
0,League of Legends,376000.0
1,Grand Theft Auto V,285000.0
2,VALORANT,211000.0
3,Rust,128000.0
4,Fortnite,122000.0
...,...,...
295,"Animals, Aquariums, and Zoos",3000.0
296,TrackMania,3000.0
297,Tibia,2900.0
298,Raft,2900.0


In [45]:
df.drop(df.tail(1).index,inplace=True)

In [46]:
df

,GAME,VIEWS
0,League of Legends,376000.0
1,Grand Theft Auto V,285000.0
2,VALORANT,211000.0
3,Rust,128000.0
4,Fortnite,122000.0
...,...,...
294,Street Fighter V,3100.0
295,"Animals, Aquariums, and Zoos",3000.0
296,TrackMania,3000.0
297,Tibia,2900.0


In [49]:
# Convert DataFrame into Dictionary
twitch = pd.Series(df.VIEWS.values,index=df.GAME).to_dict()

In [50]:
twitch

{'League of Legends': 376000.0,
 'Grand Theft Auto V': 285000.0,
 'VALORANT': 211000.0,
 'Rust': 128000.0,
 'Fortnite': 122000.0,
 'Apex Legends': 93200.0,
 'Escape from Tarkov': 91700.0,
 'Dota 2': 72900.0,
 'Call of Duty: Warzone': 64900.00000000001,
 'Minecraft': 57700.0,
 'FIFA 22': 52000.0,
 'Counter-Strike: Global Offensive': 50000.0,
 'Slots': 47600.0,
 'Monster Hunter Rise': 35400.0,
 'Music': 34400.0,
 'Teamfight Tactics': 33700.0,
 'God of War': 32400.0,
 'Dead by Daylight': 31800.0,
 'Hearthstone': 30200.0,
 '2': 29000.0,
 'Special Events': 26700.0,
 'LOST ARK': 24300.0,
 'Rocket League': 22400.0,
 'PUBG: BATTLEGROUNDS': 21900.0,
 'Sports': 21600.0,
 'Genshin Impact': 20700.0,
 'ASMR': 19400.0,
 'Dying Light 2: Stay Human': 17900.0,
 'Overwatch': 14900.0,
 'Art': 12900.0,
 'Final Fantasy XIV Online': 12700.0,
 'Chess': 12400.0,
 'Clash Royale': 12300.0,
 'Red Dead Redemption 2': 11900.0,
 'Talk Shows &amp; Podcasts': 11200.0,
 'Virtual Casino': 10800.0,
 'Project Zomboid': 1

# Migrating it to MongoDB

In [51]:
# connecting to mongodb
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# declare the database
db = client.twitchviews_db
collection = db.items

In [53]:
# Push into MongoDB
collection.insert_one(twitch)